In [0]:
pip install nltk

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import nltk
import re

from nltk.corpus import stopwords
from nltk.tag import pos_tag

nltk.download("stopwords")
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Out[1]: True

In [0]:
#Load the file 
littlewomen = sc.textFile("/FileStore/tables/Little_Women.txt")
littlewomen.take(20)

Out[2]: ['The Project Gutenberg eBook of Little Women, by Louisa May Alcott',
 '',
 'This eBook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. You may copy it, give it away or re-use it under the terms',
 'of the Project Gutenberg License included with this eBook or online at',
 'www.gutenberg.org. If you are not located in the United States, you',
 'will have to check the laws of the country where you are located before',
 'using this eBook.',
 '',
 'Title: Little Women',
 '',
 'Author: Louisa May Alcott',
 '',
 'Release Date: May, 1996 [eBook #514]',
 '[Most recently updated: November 27, 2021]',
 '',
 'Language: English',
 '',
 '']

In [0]:
#split the file into words of length greater than 0
littlewomen_split = littlewomen.flatMap(lambda x: re.split("[\[\]# ./\"'-:;|?*“”’‘]", x)).filter(lambda x: len(x) > 0)
littlewomen_split.take(15)

Out[3]: ['The',
 'Project',
 'Gutenberg',
 'eBook',
 'of',
 'Little',
 'Women',
 'by',
 'Louisa',
 'May',
 'Alcott',
 'This',
 'eBook',
 'is',
 'for']

In [0]:
#filter stopwords using function from nltk library
littlewomen_filtered = littlewomen_split.filter(lambda x: x.lower() not in stopwords.words("english"))
littlewomen_filtered.take(15)

---------------------------------------------------------------------------
PicklingError                             Traceback (most recent call last)
<command-4463470365468534> in <module>
      1 #filter stopwords using function from nltk library
      2 littlewomen_filtered = littlewomen_split.filter(lambda x: x.lower() not in stopwords.words("english"))
----> 3 littlewomen_filtered.take(15)

/databricks/spark/python/pyspark/rdd.py in take(self, num)
   1598 
   1599             p = range(partsScanned, min(partsScanned + numPartsToTry, totalParts))
-> 1600             res = self.context.runJob(self, takeUpToNumLeft, p)
   1601 
   1602             items += res

/databricks/spark/python/pyspark/context.py in runJob(self, rdd, partitionFunc, partitions, allowLocal)
   1333             finally:
   1334                 os.remove(filename)
-> 1335         sock_info = self._jvm.PythonRDD.runJob(self._jsc.sc(), mappedRDD._jrdd, partitions)
   1336         return list(_load_from_socket(soc

In [0]:
#filter only the text where part of speech is a noun
named_entities = littlewomen_filtered.filter(lambda x: nltk.pos_tag([x])[0][1] == "NN")
named_entities.take(15)

Out[5]: ['Project',
 'eBook',
 'Alcott',
 'eBook',
 'use',
 'anyone',
 'world',
 'cost',
 'whatsoever',
 'copy',
 'use',
 'Project',
 'License',
 'eBook',
 'online']

In [0]:
#make a map out of the words and their appearances and then aggregate to count their frequencies, sorting in descending order
sorted_freqs = named_entities.map(lambda x: (x.lower(), 1)).reduceByKey(lambda x,y:x+y).sortBy(lambda x: -x[1])
sorted_freqs.take(15) 

Out[6]: [('jo', 1347),
 ('meg', 682),
 ('amy', 647),
 ('laurie', 593),
 ('mother', 368),
 ('time', 321),
 ('mr', 310),
 ('think', 275),
 ('home', 269),
 ('face', 252),
 ('dear', 248),
 ('mrs', 244),
 ('look', 235),
 ('day', 235),
 ('way', 213)]